In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing, metrics, dummy, linear_model, ensemble
from xgboost import XGBClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
import shap

## Part 1

## Data Wrangling

In [3]:
#load the data and insepect the first few rows

path = 'data_science_assignment.csv'

df = pd.read_csv(path)

df.head()

,Success,Model_B,Date,X1,X2,X3,X4
0,0,0.320440,2018-07,0,-0.897088,0.804294,0.707665
1,0,0.491581,2018-09,1,-0.328084,-0.332256,-0.340870
2,0,0.085481,2015-08,0,1.272140,2.243532,0.815581
3,1,0.839927,2017-07,1,0.290594,-1.161187,-0.881761
4,0,0.146804,2018-09,0,-0.307462,1.161276,1.597084


In [5]:
#convert the Date field from object to datetime
df.Date = pd.to_datetime(df.Date)

In [9]:
#month
df['month'] = df.Date.dt.month
df.month

0        7
1        9
2        8
3        7
4        9
        ..
5995     2
5996    11
5997     4
5998    12
5999     4
Name: month, Length: 6000, dtype: int64

Success rates are highest in the 1st quarter of the year. They tend to be lowest in the 3rd quarter and moderate in quarters 2 and 4. A feature can be created to reflect the quarter.

In [10]:
#function to map month to quarter
def map_month_quarter(row):
    if row['month'] <= 3:
        return 'Q1'
    elif row['month'] >3 and row['month'] <=6:
        return 'Q2'
    elif row['month'] >6 and row['month'] <=9:
        return 'Q3'
    else:
        return 'Q4'

In [11]:
#map each month to a quarter
df['quarter'] = df.apply(lambda row : map_month_quarter(row), axis=1)

In [12]:
#drop the Date column
df.drop(columns=['Date'],inplace=True)

In [13]:
#set a value from random state to ensure repeatability
random_state=42

In [188]:
#get train/test data when X1 = True
X_train_X1_true = X_train[df['X1'] == 1]
y_train_X1_true = y_train[X_train_X1_true.index]
X_test_X1_true = X_test[df['X1'] == 1]
y_test_X1_true = y_test[X_test_X1_true.index]

#get train/test data when X1 = False
X_train_X1_false = X_train[df['X1'] == 0]
y_train_X1_false = y_train[X_train_X1_false.index]
X_test_X1_false = X_test[df['X1'] == 0]
y_test_X1_false = y_test[X_test_X1_false.index]

Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.
Boolean Series key will be reindexed to match DataFrame index.


In [193]:
rf_model_X1_True = ensemble.RandomForestClassifier(random_state=random_state)

dict_score = train_predict_score(rf_model_X1_True, X_train_X1_true, y_train_X1_true, 
                                 X_test_X1_true, y_test_X1_true)
rf_model_X1_False = ensemble.RandomForestClassifier(random_state=random_state)

dict_score = train_predict_score(rf_model_X1_False, X_train_X1_false, y_train_X1_false, 
                                 X_test_X1_false, y_test_X1_false)

In [194]:
df_x1_true_predictions = pd.DataFrame(rf_model_X1_True.predict_proba(X_test_X1_true)[:,1])
df_x1_true_predictions.index = y_test_X1_true.index
df_x1_true_predictions['Actual'] = y_test_X1_true

#rename the predictions column
df_x1_true_predictions.rename(columns={0:'Model_A_Prob'}, inplace=True)

#Model A (segment X1 = False) predictions and actuals
df_x1_false_predictions = pd.DataFrame(rf_model_X1_False.predict_proba(X_test_X1_false)[:,1])
df_x1_false_predictions.index = y_test_X1_false.index
df_x1_false_predictions['Actual'] = y_test_X1_false

#rename the predictions column
df_x1_false_predictions.rename(columns={0:'Model_A_Prob'}, inplace=True)

#stack the dataframes into a single set of of predictions/actuals for Model A
df_actual_pred = pd.concat([df_x1_true_predictions, df_x1_false_predictions])

#add the model B predictions
df_actual_pred = pd.concat([df_actual_pred, X_test.Model_B], axis=1)
df_actual_pred = df_actual_pred.rename(columns={'Model_B':'Model_B_Prob'})

In [195]:
df_actual_pred = df_actual_pred.sort_values(by='Model_A_Prob')
df_actual_pred['Rank_A'] = range(1,len(df_actual_pred) + 1)

#cut the dataframe into 10 equal sized buckets (based on Model A Predictions)
df_actual_pred['Bin'] = pd.qcut(df_actual_pred['Rank_A'],10)

#group the data in order to inspect profit/loss for each bin/group
df_grouped_data_A = df_actual_pred.groupby('Bin').agg({'Actual':'sum', 'Model_A_Prob':'count'}) \
    .rename(columns={'Model_A_Prob':'Count', 'Actual':'Successes'})

#determine the cost of the emails and the value provided by the emails for each group/bin
df_grouped_data_A['Cost'] = df_grouped_data_A['Count'] * 0.01
df_grouped_data_A['Value'] = df_grouped_data_A['Successes'] * 0.25

#determine profit or loss for each group/bin
df_grouped_data_A['P/L'] = df_grouped_data_A['Value'] - df_grouped_data_A['Cost']

df_actual_pred.drop(columns=['Rank_A','Bin'], inplace=True)

In [14]:
#separate X (feature matrix) and y (target vector)
X = df.drop(columns=['Success'])
y = df.Success

In [163]:
#train/test split
X_train, X_test, y_train, y_test = \
    model_selection.train_test_split(X, y, train_size=0.7, stratify=y, random_state=random_state)

**Categorical Encoding**

In [164]:
#encode categorical columns using One Hot Encoding

#get the categorical columns from the training data
categorical_columns = ['month', 'quarter']
df_categorical = X_train[categorical_columns]

#initialize the One Hot encoder
encoder = preprocessing.OneHotEncoder(sparse=False)

#fit the encoder to the training data, encode the training data, add the encoded columns to the dataframe
encoded_cols = encoder.fit_transform(df_categorical)
df_encoded = pd.DataFrame(encoded_cols, 
                          columns=encoder.get_feature_names(categorical_columns),
                         index=X_train.index)
X_train = pd.concat([X_train,df_encoded], axis=1)

#drop the original categorical columns
X_train.drop(columns=categorical_columns, inplace=True)


#get the categorical columns from the test data
df_categorical = X_test[categorical_columns]

#encode the test data using the encoder that was fitted to the training data
encoded_cols = encoder.transform(df_categorical)
df_encoded = pd.DataFrame(encoded_cols, 
                          columns=encoder.get_feature_names(categorical_columns),
                         index=X_test.index)
X_test = pd.concat([X_test,df_encoded], axis=1)

#drop the original categorical columns
X_test.drop(columns=categorical_columns, inplace=True)

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.


**Typically the numerical columns would be scaled. However, this data already appears to have been scaled so the scaling step can be skipped**

## Multicollinearity

**Remove the Feature With the Greatest VIF Until All Features Have VIF < 2:**

In [165]:
#We want to keep the Model_B feature. Therefore, we will remove it from the dataframe, remove multicollinear features,
#and then add Model_B back to the dataframe.
X_train_model_B = X_train['Model_B']
X_test_model_B = X_test['Model_B']

X_train.drop(columns=['Model_B'], inplace=True)
X_test.drop(columns=['Model_B'], inplace=True)

In [166]:
X_train = X_train.sort_index(axis=1)
X_test = X_test.sort_index(axis=1)

while len(X_train.columns) > 0:

    df_VIF = pd.DataFrame({'Feature':X_train.columns})
    df_VIF['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(len(X_train.columns))]
    
    if df_VIF.loc[df_VIF['VIF'].idxmax()]['VIF'] < 2:
        break
        
    X_train = X_train.drop(columns=df_VIF.loc[df_VIF['VIF'].idxmax()]['Feature'])   

divide by zero encountered in double_scalars
divide by zero encountered in double_scalars
divide by zero encountered in double_scalars
divide by zero encountered in double_scalars


In [167]:
X_train['Model_B'] = X_train_model_B 
X_test['Model_B'] = X_test_model_B 

**Select the same columns from the test data**

In [170]:
#Select the same columns from the test data
X_test = X_test[X_train.columns]

In [169]:
X_train.head()

,X2,X3,month_11,month_12,month_2,month_3,month_5,month_6,month_8,month_9,Model_B
2062,-0.935495,-1.656392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.765860
1966,0.754162,0.162420,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.320886
4758,1.681657,2.247262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.219703
1949,-0.006841,-0.064094,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.770482
1891,0.188642,0.828965,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.347914


In [149]:
X_train.head()

,X2,X3,month_11,month_12,month_2,month_3,month_5,month_6,month_8,month_9
2062,-0.935495,-1.656392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1966,0.754162,0.162420,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4758,1.681657,2.247262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1949,-0.006841,-0.064094,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1891,0.188642,0.828965,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


**Add the Model_B columns back into the dataset**

In [53]:
#add the model B columns back to X_train and X_test
X_train['Model_B'] = X_train_model_B 
X_test['Model_B'] = X_test_model_B 

In [150]:
#method to train, predict, and score each model
def train_predict_score(model, X_train, y_train, X_test, y_test):
    model.fit(X_train,y_train)
    
    #metrics on training data
    train_accuracy = metrics.accuracy_score(y_train,model.predict(X_train))
    train_auc = metrics.roc_auc_score(y_train,model.predict_proba(X_train)[:,1])
    train_f1 = metrics.f1_score(y_train,model.predict(X_train))
    train_recall = metrics.recall_score(y_train, model.predict(X_train))
    train_precision = metrics.precision_score(y_train, model.predict(X_train))
    
    #metrics on test data
    test_accuracy = metrics.accuracy_score(y_test,model.predict(X_test))
    test_auc = metrics.roc_auc_score(y_test,model.predict_proba(X_test)[:,1])
    test_f1 = metrics.f1_score(y_test,model.predict(X_test))
    test_recall = metrics.recall_score(y_test, model.predict(X_test))
    test_precision = metrics.precision_score(y_test, model.predict(X_test))
      
    return dict({
        'Train Accuracy':train_accuracy,
        'Train AUC': train_auc,
        'Train F1': train_f1,
        'Train Recall': train_recall,
        'Train Precision': train_precision,
        'Test Accuracy':test_accuracy,
        'Test AUC':test_auc,
        'Test F1':test_f1,
        'Test Recall': test_recall,
        'Test Precision': test_precision})

In [171]:
#DataFrame to hold scores
model_scores = pd.DataFrame(columns=['Model','Train Accuracy','Train AUC','Train F1', 'Train Recall', 
                                     'Train Precision', 'Test Accuracy','Test AUC', 'Test F1',
                                     'Test Recall', 'Test Precision'])

In [172]:
#initial logistic regression model
lr_model = linear_model.LogisticRegression(random_state=random_state)

dict_score = train_predict_score(lr_model, X_train, y_train, X_test, y_test)
dict_score['Model'] = 'Logistic Regression'

model_scores = model_scores.append(dict_score, ignore_index=True)

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [173]:
#initial random forest model
rf_model = ensemble.RandomForestClassifier(random_state=random_state)

dict_score = train_predict_score(rf_model, X_train, y_train, X_test, y_test)
dict_score['Model'] = 'Random Forest'

model_scores = model_scores.append(dict_score, ignore_index=True)

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [97]:
#inspect the initial model scores
display(model_scores.sort_values(by='Test Recall', ascending=False))

,Model,Train Accuracy,Train AUC,Train F1,Train Recall,Train Precision,Test Accuracy,Test AUC,Test F1,Test Recall,Test Precision
1,Random Forest,1.0,1.0,1.0,1.0,1.0,0.948889,0.974854,0.836299,0.839286,0.833333
0,Logistic Regression,0.936429,0.976972,0.780968,0.727829,0.842478,0.944444,0.980458,0.814126,0.782143,0.848837


In [174]:
#inspect the initial model scores
display(model_scores.sort_values(by='Test Recall', ascending=False))

,Model,Train Accuracy,Train AUC,Train F1,Train Recall,Train Precision,Test Accuracy,Test AUC,Test F1,Test Recall,Test Precision
1,Random Forest,1.0,1.0,1.0,1.0,1.0,0.948889,0.974854,0.836299,0.839286,0.833333
0,Logistic Regression,0.936429,0.976972,0.780968,0.727829,0.842478,0.944444,0.980458,0.814126,0.782143,0.848837


In [181]:
#get each model's predictions
rf_predictions = (rf_model.predict_proba(X_test)[:,1] > 0.15).astype(np.int64)
#rf_x1_true_predictions = (rf_model_X1_True.predict_proba(X_test_X1_true)[:,1] > 0.5).astype(np.int64)
#rf_x1_false_predictions = (rf_model_X1_False.predict_proba(X_test_X1_false)[:,1] > 0.5).astype(np.int64)

In [182]:
pd.crosstab(y_test,rf_predictions)

col_0,0,1
Success,,
0,1391,129
1,19,261


In [183]:
rf_successes = ((rf_predictions == y_test) & (y_test == 1)).sum()
rf_fp = ((rf_predictions != y_test) & (rf_predictions == 1)).sum()
(rf_successes * 0.24) - (rf_fp * 0.01)

61.35

In [137]:
B_predictions = (X_test.Model_B > 0.5).astype(np.int64)
B_successes = ((B_predictions == y_test) & (y_test == 1)).sum()
B_fp = ((B_predictions != y_test) & (B_predictions == 1)).sum()
(B_successes * 0.24) - (B_fp * 0.01)

61.5

In [138]:
#get the model B predictions
Model_B_predictions = (X_test.Model_B > 0.5).astype(np.int64)

#get the number of true positives for Model A vs. Model B
model_A_tp = rf_successes
model_B_tp = ((Model_B_predictions == y_test) & (y_test == 1)).sum()

#calculate recall for model B
model_B_fn = ((Model_B_predictions != y_test) & (Model_B_predictions == 0)).sum()
model_B_recall = model_B_tp / (model_B_tp + model_B_fn)

#calculate recall for model A
rf_x1_true_fn = ((rf_predictions != y_test) & (rf_predictions == 0)).sum()
rf_x1_false_fn = ((rf_predictions != y_test) & (rf_predictions == 0)).sum()
model_A_fn = rf_x1_true_fn + rf_x1_false_fn

model_A_recall = rf_successes / (rf_successes + model_A_fn)

#DataFrame to hold correct prediction count
df_model_comparison = pd.DataFrame(columns=['Model','True Positives','Recall'])
df_model_comparison = \
    df_model_comparison.append(dict({'Model':'A', 'True Positives': model_A_tp, 
                                 'Recall':model_A_recall}), ignore_index=True)
df_model_comparison = \
    df_model_comparison.append(dict({'Model':'B', 'True Positives': model_B_tp, 
                                 'Recall':model_B_recall}), ignore_index=True)

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [139]:
df_model_comparison

,Model,True Positives,Recall
0,A,269,0.924399
1,B,280,1.0


**Model B outperformed Model A in terms of both True Positives and Recall**

## Part 4

In [158]:
#Model A (segment X1 = True) predctions and actuals
df_x1_true_predictions = pd.DataFrame(rf_model.predict_proba(X_test)[:,1])
df_x1_true_predictions.index = y_test.index
df_x1_true_predictions['Actual'] = y_test

#rename the predictions column
df_x1_true_predictions.rename(columns={0:'Model_A_Prob'}, inplace=True)



#add the model B predictions
df_actual_pred = pd.concat([df_x1_true_predictions, X_test.Model_B], axis=1)
df_actual_pred = df_actual_pred.rename(columns={'Model_B':'Model_B_Prob'})

<class 'AttributeError'>: 'DataFrame' object has no attribute 'Model_B'

In [184]:
#Model A (segment X1 = True) predctions and actuals
df_x1_true_predictions = pd.DataFrame(rf_model.predict_proba(X_test)[:,1])
df_x1_true_predictions.index = y_test.index
df_x1_true_predictions['Actual'] = y_test

#rename the predictions column
df_x1_true_predictions.rename(columns={0:'Model_A_Prob'}, inplace=True)



#add the model B predictions
df_actual_pred = df_x1_true_predictions


In [185]:
df_actual_pred.describe()

,Model_A_Prob,Actual
count,1800.000000,1800.000000
mean,0.158767,0.155556
std,0.309317,0.362534
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.080000,0.000000
max,1.000000,1.000000


## Model A

In [186]:
#since Model A predicted a lot of 0s we cannot use qcut on the 'Model_A_Prob' column,
#therefore we can sort the dataframe and use rank values
df_actual_pred = df_actual_pred.sort_values(by='Model_A_Prob')
df_actual_pred['Rank_A'] = range(1,len(df_actual_pred) + 1)

#cut the dataframe into 10 equal sized buckets (based on Model A Predictions)
df_actual_pred['Bin'] = pd.qcut(df_actual_pred['Rank_A'],10)

#group the data in order to inspect profit/loss for each bin/group
df_grouped_data_A = df_actual_pred.groupby('Bin').agg({'Actual':'sum', 'Model_A_Prob':'count'}) \
    .rename(columns={'Model_A_Prob':'Count', 'Actual':'Successes'})

#determine the cost of the emails and the value provided by the emails for each group/bin
df_grouped_data_A['Cost'] = df_grouped_data_A['Count'] * 0.01
df_grouped_data_A['Value'] = df_grouped_data_A['Successes'] * 0.25

#determine profit or loss for each group/bin
df_grouped_data_A['P/L'] = df_grouped_data_A['Value'] - df_grouped_data_A['Cost']

df_actual_pred.drop(columns=['Rank_A','Bin'], inplace=True)

## Model B

In [145]:
#cut the dataframe into 10 equal sized buckets (based on Model A Predictions)
df_actual_pred['Bin'] = pd.qcut(df_actual_pred['Model_B_Prob'],10)

#group the data in order to inspect profit/loss for each bin/group
df_grouped_data_B = df_actual_pred.groupby('Bin').agg({'Actual':'sum', 'Model_B_Prob':'count'}) \
    .rename(columns={'Model_B_Prob':'Count', 'Actual':'Successes'})

#determine the cost of the emails and the value provided by the emails for each group/bin
df_grouped_data_B['Cost'] = df_grouped_data_B['Count'] * 0.01
df_grouped_data_B['Value'] = df_grouped_data_B['Successes'] * 0.25

#determine profit or loss for each group/bin
df_grouped_data_B['P/L'] = df_grouped_data_B['Value'] - df_grouped_data_B['Cost']

df_actual_pred.drop(columns='Bin', inplace=True)

In [196]:
df_grouped_data_A

,Successes,Count,Cost,Value,P/L
Bin,,,,,
"(0.999, 180.9]",1,180,1.8,0.25,-1.55
"(180.9, 360.8]",0,180,1.8,0.00,-1.80
"(360.8, 540.7]",0,180,1.8,0.00,-1.80
"(540.7, 720.6]",0,180,1.8,0.00,-1.80
"(720.6, 900.5]",1,180,1.8,0.25,-1.55
"(900.5, 1080.4]",0,180,1.8,0.00,-1.80
"(1080.4, 1260.3]",3,180,1.8,0.75,-1.05
"(1260.3, 1440.2]",18,180,1.8,4.50,2.70
"(1440.2, 1620.1]",91,180,1.8,22.75,20.95


In [147]:
df_grouped_data_B

,Successes,Count,Cost,Value,P/L
Bin,,,,,
"(0.0219, 0.21]",0,180,1.8,0.00,-1.80
"(0.21, 0.291]",0,180,1.8,0.00,-1.80
"(0.291, 0.358]",0,180,1.8,0.00,-1.80
"(0.358, 0.42]",0,180,1.8,0.00,-1.80
"(0.42, 0.483]",0,180,1.8,0.00,-1.80
"(0.483, 0.547]",0,180,1.8,0.00,-1.80
"(0.547, 0.618]",4,180,1.8,1.00,-0.80
"(0.618, 0.696]",18,180,1.8,4.50,2.70
"(0.696, 0.788]",91,180,1.8,22.75,20.95
